# Graph Neural Networks

## Why GNNs?

Answer briefly:

1. Consider the given list of limitations of shallow encoders like random walk based node embedding methods. How do GNNs solve them?
    - Poor scalability ($|V|d$ parameters are needed, $d$: dimension of the embedding space)
    - Transductive nature (Cannot obtain embeddings for nodes not in the training set.)
    - Cannot capture structural similarity
    - Cannot utilize node, edge, and graph features

1. Given an undirected graph $G=(V,E)$, let's flatten the adjacency matrix $A$ (i.e., concatenate the rows into a single vector) and feed it to a Multi-Layer Perceptron (MLP). What's wrong with this approach?

1. In a CNN, a convolutional kernel aggregates information from local pixel neighborhoods. What would "locality" mean on a graph? How could we define a convolution operation that respects this locality?

## Permutation Invariance and Equivariance

1. Consider a graph with three node labels A, B, C and the adjacency matrix:
    $$
        A = \begin{bmatrix}
        0 & 1 & 1\\
        1 & 0 & 0\\
        1 & 0 & 0
        \end{bmatrix}
    $$

    Suppose we feed this graph into a GNN layer defined as:
    $$
        h_v^{(1)} = \sigma (W \cdot \text{AGG}(\{h_u^{(0)}: u \in N(v)\}))
    $$

    where AGG is the **sum** function.

    If we permute the node order to (C, A, B), will the node embeddings change after the layer? Why or why not?

2. Let $A \in \{0,1\}^{n \times n}$ be the adjacency matrix and $X \in \mathbb{R}^{n \times d}$ be the node feature matrix. Let $P$ be an $n \times n$ permutation matrix (it reorders node indices). Permutation of the graph means:
    $$
        A^\prime = PAP^\intercal, \quad X^\prime=PX
    $$

    For each function $f(A,X)$ below, determine wheteher it is:
    - Permutation invariant: $f(A^\prime ,X^\prime) = f(A,X)$
    - Permutation equivariant: $f(A^\prime ,X^\prime) = Pf(A,X)$
    - Neither

    | Function                             | Inv./Equiv./Neither |
    |--------------------------------------|---------------------|
    | $f(A,X) = 1^\intercal X$             |                     |
    | $f(A,X) = X$                         |                     |
    | $f(A,X) = AX$                        |                     |
    | $f(A,X) = A^\intercal X$             |                     |
    | $f(A,X) = \text{ReLU}(AXW)$          |                     |
    | $f(A,X) = \frac{1}{n} 1^\intercal X$ |                     |
    | $f(A,X) = X^\intercal X$             |                     |
    | $f(A,X) = XW$                        |                     |
    | $f(A,X) = A_{1,:}X$                  |                     |
    | $f(A,X) = \text{sort}(X)$            |                     |

## One GNN Layer

Consider the following simple undirected graph $G=(V,E)$:
$$
    V=\{ 1,2,3 \}, \quad E=\{ \{1,2\},\{2,3\} \}
$$

The initial node feature matrix and the (unweighted) adjacency matrix are given as follows:
$$
    X = 
    \begin{bmatrix}
        1 & 0 \\
        0 & 1 \\
        1 & 1 
    \end{bmatrix}, \quad
    A = 
    \begin{bmatrix}
        0 & 1 & 0 \\
        1 & 0 & 1 \\
        0 & 1 & 0 
    \end{bmatrix}
$$

We apply one Graph Convolutional Network (GCN) layer as defined by Kipf & Welling (2017):
$$
    H = \sigma(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}XW)
$$
where 

- $D$ is the degree matrix ($D=\text{diag}(1,2,1)$)
- $\tilde{A}=A+I$
- $\tilde{D}_{ii}=\sum_j \tilde{A}_{ij}$
- $\sigma(\cdot)$ is ReLU

and for simplicity, the weight matrix is 
$$
    W = 
    \begin{bmatrix}
        1 & 0 \\
        0 & 1 \\
    \end{bmatrix}
$$

Tasks:

1. Compute $\tilde{A}$ and $\tilde{D}$
1. Compute the normalized adjacency $\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}$
1. Multiply with $XW$
1. Apply the ReLU
1. Write down the resulting node embeddings $H_1,H_2,H_3$
1. Now compare the initial node features $X$ and the embeddings $H$. How did each node's location in the embedding space change and why?

## Programming: One GNN Layer with Torch

In this exercise, you'll implement the same task given in the previous example with `torch`. 

1. Complete the following code snippet and reproduce your result.

In [43]:
import torch

# initial features
X = torch.tensor([
    [1., 0.],  # Node 1
    [0., 1.],  # Node 2
    [1., 1.]   # Node 3
])

# toy graph: 1–2–3
A = torch.tensor([
    [0., 1., 0.],
    [1., 0., 1.],
    [0., 1., 0.]
])

# weight matrix (identity for simplicity)
W = torch.eye(2)

# Task 1: Compute A_hat and D_hat
I = torch.eye(3)
A_hat = A + I
D_hat = torch.diag(A_hat.sum(dim=1))

# Task 2: Compute normalized adjacency
D_hat_inv_sqrt = torch.linalg.inv(torch.sqrt(D_hat))
A_norm = D_hat_inv_sqrt @ A_hat @ D_hat_inv_sqrt

# Task 3-4: Multiply normalized adjacency with XW and apply ReLU
H = torch.relu(A_norm @ X @ W)

print("Initial features X:\n", X)
print("\nNormalized adjacency A_norm:\n", A_norm)
print("\nEmbeddings H:\n", H)


Initial features X:
 tensor([[1., 0.],
        [0., 1.],
        [1., 1.]])

Normalized adjacency A_norm:
 tensor([[0.5000, 0.4082, 0.0000],
        [0.4082, 0.3333, 0.4082],
        [0.0000, 0.4082, 0.5000]])

Embeddings H:
 tensor([[0.5000, 0.4082],
        [0.8165, 0.7416],
        [0.5000, 0.9082]])


2. Now, re-apply the GCN layer 10 more times. What happens to the embeddings? Interpret your results.


In [ ]:
for k in range(10):
    H = torch.relu(A_norm @ H @ W)
    print(f"\nAfter {k} layers:\n", H)


After 0 layers:
 tensor([[0.5833, 0.5069],
        [0.6804, 0.7847],
        [0.5833, 0.7569]])

After 1 layers:
 tensor([[0.5694, 0.5738],
        [0.7031, 0.7775],
        [0.5694, 0.6988]])

After 2 layers:
 tensor([[0.5718, 0.6043],
        [0.6993, 0.7787],
        [0.5718, 0.6668]])

After 3 layers:
 tensor([[0.5714, 0.6200],
        [0.6999, 0.7785],
        [0.5714, 0.6513]])

After 4 layers:
 tensor([[0.5714, 0.6278],
        [0.6998, 0.7785],
        [0.5714, 0.6435]])

After 5 layers:
 tensor([[0.5714, 0.6317],
        [0.6999, 0.7785],
        [0.5714, 0.6395]])

After 6 layers:
 tensor([[0.5714, 0.6337],
        [0.6999, 0.7785],
        [0.5714, 0.6376]])

After 7 layers:
 tensor([[0.5714, 0.6347],
        [0.6999, 0.7785],
        [0.5714, 0.6366]])

After 8 layers:
 tensor([[0.5714, 0.6352],
        [0.6999, 0.7785],
        [0.5714, 0.6361]])

After 9 layers:
 tensor([[0.5714, 0.6354],
        [0.6999, 0.7785],
        [0.5714, 0.6359]])
